<a href="https://colab.research.google.com/github/CoAxLab/Data-Explorations/blob/main/book/exercises/power-analysis-via-simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

1. Simulating data 1/1
2. run_analysis function 2/2
3. repeat_analysis() function 3/3
4. Testing different sample sizes 2/2
5. Reflection 2/2

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [ ]:
sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
  # Generate ages uniformly between age_lo and age_hi
  age <- runif(sample_size, age_lo, age_hi)

  # Generate word reading experience (x) as a function of age plus noise
  x <- beta_x0 + beta_xa * age + rnorm(sample_size, 0, sd_x)

  # Generate reading comprehension (c) as a function of age and experience plus noise
  c <- beta_c0 + beta_ca * age + beta_cx * x + rnorm(sample_size, 0, sd_c)

      return(data.frame(age=age,x=x,c=c)) # it's actually bad form to have a variable named "c" in R, my bad...
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,124.71148,28.839149,313.17540
2,128.27629,99.070617,597.38314
3,127.37073,4.527544,86.72925
4,177.69228,119.463737,582.83518
5,185.54174,167.661905,634.92714
6,88.46584,25.389679,93.24581


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [ ]:
# Run Mediation Analysis
# library(mediation)
run_analysis <- function(dat) {
  # Fit the mediator model (x ~ age)
  fit_mediator <- lm(x ~ age, data = dat)

  # Fit the outcome model (c ~ age + x)
  fit_outcome <- lm(c ~ age + x, data = dat)

  # Run the mediation analysis
  fitMed <- mediate(fit_mediator, fit_outcome, treat = "age", mediator = "x",
                   boot = TRUE, sims = 1000)

  # Return ACME and ADE estimates and p-values
  return(c(
    ACME_est = fitMed$d0,      # Average Causal Mediation Effect estimate
    ACME_p = fitMed$d0.p,      # ACME p-value
    ADE_est = fitMed$z0,       # Average Direct Effect estimate
    ADE_p = fitMed$z0.p        # ADE p-value
  ))
}

# Test the function on our simulated data
run_analysis(dat)


Running nonparametric bootstrap




ACME_est   ACME_p  ADE_est    ADE_p 
1.409354 0.004000 1.196452 0.002000

---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [ ]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    colnames(simouts) <- c("ACME_est", "ACME_p", "ADE_est", "ADE_p")

    # Start simulating
    for (i in 1:num_simulations) {
      # Simulate new data
      sim_dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x,
                             beta_ca, beta_cx, beta_c0, sd_c)

      # Run analysis and store results
      simouts[i, ] <- run_analysis(sim_dat)
    }

    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[, "ACME_p"] < alpha)
    ADE_cov = mean(simouts[, "ADE_p"] < alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

In [ ]:
# Run the repeat_analysis function with 10 simulations and alpha = 0.01
set.seed(123) # For reproducibility
results_10 <- repeat_analysis(10, 0.01, sample_size, age_lo, age_hi,
                            beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
print(results_10$ACME_cov)
print(results_10$ADE_cov)

Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap




[1] 1
[1] 0.6


---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [ ]:
# Define sample sizes to test
sample_sizes <- c(50, 75, 100, 125, 150)

# Initialize results matrix
power_results <- matrix(NA, nrow = length(sample_sizes), ncol = 2)
colnames(power_results) <- c("ACME_power", "ADE_power")
rownames(power_results) <- paste0("n=", sample_sizes)

# Run simulations for each sample size
for (i in 1:length(sample_sizes)) {
  set.seed(123 + i) # Different seed for each sample size
  res <- repeat_analysis(10, 0.01, sample_sizes[i], age_lo, age_hi,
                        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

  power_results[i, "ACME_power"] <- res$ACME_cov
  power_results[i, "ADE_power"] <- res$ADE_cov
}

Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonpar

Print your results.

In [ ]:
# Print results
power_results

,ACME_power,ADE_power
n=50,0.3,0.4
n=75,0.5,0.7
n=100,0.6,0.4
n=125,0.8,1.0
n=150,1.0,1.0


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> *Based on our results it appears that n=125 is a sufficient sample size for large effect sizes based on our power analysis.*
>

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *This was likely because the magnitude of the direct effect is smaller than the indirect effect, and the increased influence of noise makes it harder to detect across different sample sizes, especially with the limited number of simulations*

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*